In [129]:
import time
a=time.perf_counter()

In [130]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, date 
from lightfm.data import Dataset
from lightfm import LightFM
from lightfm import cross_validation
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
import random

In [131]:
path='./data_juin/'
df_user=pd.read_csv(path + 'user.csv',sep=',')
df_favorite=pd.read_csv(path + 'favorite.csv',sep=',')
df_visit=pd.read_csv(path + 'visit.csv',sep=',')
df_place=pd.read_csv(path + 'place.csv',sep=',',encoding='latin-1')
review=pd.read_csv(path + 'review.csv',sep=',',encoding='latin-1')
df_place_place_type=pd.read_csv(path + 'place_place_type.csv',sep=',')
place_type=pd.read_csv(path + 'place_type.csv',sep=',')

In [132]:
def generate_int_id(dataframe, id_col_name):
    """
    Generate unique integer id for users, questions and answers

    Parameters
    ----------
    dataframe: Dataframe
        Pandas Dataframe for Users or Q&A. 
    id_col_name : String 
        New integer id's column name.
        
    Returns
    -------
    Dataframe
        Updated dataframe containing new id column 
    """
    new_dataframe=dataframe.assign(
        int_id_col_name=np.arange(len(dataframe))
        ).reset_index(drop=True)
    return new_dataframe.rename(columns={'int_id_col_name': id_col_name})

#drop columns if they have to many na 

def drop_columns_na(dataframe,pourcentna):
    for i in dataframe.columns:
        pourcent=(dataframe[i].isna().sum()/dataframe[i].isna().count())
        if(pourcentna<pourcent):
            dataframe.drop(i,axis=1,inplace=True)
    return dataframe


def create_features(dataframe, features_name, id_col_name):
    """
    Generate features that will be ready for feeding into lightfm

    Parameters
    ----------
    dataframe: Dataframe
        Pandas Dataframe which contains features
    features_name : List
        List of feature columns name avaiable in dataframe
    id_col_name: String
        Column name which contains id of the question or
        answer that the features will map to.
        There are two possible values for this variable.
        1. questions_id_num
        2. professionals_id_num

    Returns
    -------
    Pandas Series
        A pandas series containing process features
        that are ready for feed into lightfm.
        The format of each value
        will be (user_id, ['feature_1', 'feature_2', 'feature_3'])
        Ex. -> (1, ['military', 'army', '5'])
    """

    features = dataframe[features_name].apply(
        lambda x: ','.join(x.map(str)), axis=1)
    features = features.str.split(',')
    features = list(zip(dataframe[id_col_name], features))
    return features



def generate_feature_list(dataframe, features_name):
    """
    Generate features list for mapping 

    Parameters
    ----------
    dataframe: Dataframe
        Pandas Dataframe for Users or Q&A. 
    features_name : List
        List of feature columns name avaiable in dataframe. 
        
    Returns
    -------
    List of all features for mapping 
    """
    features = dataframe[features_name].apply(
        lambda x: ','.join(x.map(str)), axis=1)
    features = features.str.split(',')
    features = features.apply(pd.Series).stack().reset_index(drop=True)
    return features


def calculate_auc_score(lightfm_model, interactions_matrix, 
                        question_features, professional_features): 
    """
    Measure the ROC AUC metric for a model. 
    A perfect score is 1.0.

    Parameters
    ----------
    lightfm_model: LightFM model 
        A fitted lightfm model 
    interactions_matrix : 
        A lightfm interactions matrix 
    question_features, professional_features: 
        Lightfm features 
        
    Returns
    -------
    String containing AUC score 
    """
    score = auc_score( 
        lightfm_model, interactions_matrix, 
        item_features=question_features, 
        user_features=professional_features, 
        num_threads=4).mean()
    return score

#simple fonction pour avoir user_id_light d'un user par sont user_id    
def get_user_id_light_from_user_id(user_id):
    user_id_light=user.loc[user['user_id']==user_id].user_id_light.item()
    return user_id_light


def drop_place_deja_connue(df_prediction,user_id):
    """
    cette fonction verifie qu'elle bar un user a deja en favoris ou en visit 
    et les exclue du resultas =

    Parameters
    ----------
    df_prediction: prediction des bar trouvé par le modele 

    user_id: id provenant de schloukmap pour le user  
        
    Returns
    -------
    la liste des prediction propre
    
    
    """
    favorie = df_favorite[['place_id','user_id']]
    visit = df_visit[['place_id','user_id']]
    fav_vis = pd.concat([favorie,visit])
    #on recupere tous les couple user_id place_id des visit et favori de tous les user 


    df_do_not_recomend=fav_vis.drop_duplicates()#on enleve les doublons

    user_already_visit_favori = df_do_not_recomend[df_do_not_recomend['user_id']==user_id]
    #isole la liste des bar concernant le user 

    df_prediction = df_prediction[~df_prediction.id.isin(user_already_visit_favori.place_id)]
    #suprime les bar qui apparaise des deux coté
    return df_prediction



def recommandation_for_user_by_city(user_id,city_id):
    """
    permet de faire une recommendation a un user en la limitant a la ville ou
    il ce situe et en triant les bar fermé ou non disponible et aussie ceux deja visité
    par ce dernier

    Parameters
    ----------
    id_user: user id for lightfm

    city_id: id de la ville 
        
    Returns
    -------
    dataframe des place trié par odre de pertinance pour l'utilisateur
    
    
    """
    #on convertie le user_id utilisé par schlouk map en le light_id utilisé par le modele
    light_fm_id=get_user_id_light_from_user_id(user_id)
    user = user_id_map[light_fm_id]#donne la possition dans le mapping de l'user concerné
    list_prediction=model.predict(user, np.arange(n_items)) #donne la liste des id de bar 
    #a recommendé en renvoyant dans l'ordre chaque bar et sont score 
    
    #on crée un dataframe de nos prediction
    df_prediction = pd.DataFrame(list_prediction, columns=['score'])
    #reset de l'index pour le merge car ce dernier correspond au place_id_light
    df_prediction=df_prediction.reset_index()
    #on merge nos prediction avec les bar selont l'id light pour retrouve les infos des bars
    df_prediction=df_prediction.merge(
        place,how='inner',left_on='index',right_on='place_id_light'
    )
    #on enleve les bar fermé 
    df_prediction = df_prediction.drop(df_prediction[df_prediction.is_closed==1].index)
    df_prediction = df_prediction.drop(df_prediction[df_prediction.is_published == 0].index)
    #on filtre sur la ville ou ce trouve le user
    df_prediction = df_prediction.drop(df_prediction[df_prediction.city_id != city_id].index)
    #on tri en fonction du score les bar pour voire les qu'elle sont les plus adapté au user

    df_prediction = drop_place_deja_connue(df_prediction,user_id) 

    df_prediction=df_prediction.sort_values(by='score',ascending=False)
    
    return df_prediction


    

    

on import nos user qui sont les persone a qui on doit recommander une place (bar)
on drop les columns sans intérais pour nous comme la date de création la dernier mise a jour ect 
et on leur génere un id speciale qui servira seulement pour notre model car on a besoin d'avoir des id unique par user qui commence a 0 et sont consecutif jusqu'au dernier (comme certains user de la db ne sont pas des user rélle mais ajouté par l'equipe de schlouk map pour diverse raisson on drop les personne sans firebase uid )

In [133]:
user=df_user
# user=user.drop(['created_at','updated_at'],axis=1)
user=user.dropna(subset=['firebase_uid'])
user.rename(columns={'id':'user_id'},inplace=True)
user = generate_int_id(user,"user_id_light") # genere des id unique, consecutif de zero a la dernier personne indispensable pour lightfm
user

/var/folders/_h/1j2lys1s0g344s_0q1rbvvt00000gn/T/ipykernel_4608/3228095920.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user.rename(columns={'id':'user_id'},inplace=True)


,user_id,firebase_uid,user_id_light
0,31280,zzzZClWPVSQ22uT2SqQqQQfSPon2,0
1,33287,ZZzYwUhcuTN7WUlKTNVJ0El7Gz83,1
2,29924,zZZMZBYZKXP3ClI83zvehUXXXsp2,2
3,11040,ZzZ4AF9HvmaWo3ssjW3vAmgEacj1,3
4,32661,zzxJ4S3MRxVDxcZ71UWQv7qfUG92,4
...,...,...,...
45475,16737,00KjtvKZG3ahSKyCt3AyRyTAesG3,45475
45476,31535,00FyNO18CYSV8sLQgbpKH0kQRmP2,45476
45477,39028,00e24es9Rse3DSsByF9kgR8TPHD3,45477
45478,40048,00AUdltem0aniOvFw8iRpCgO9Fo1,45478


on récupere seulement les infos qui pourrais nous etre utile pour chaque bar et comme pour les user on leur crée un lightfm id qui servira au mapping 
on modifie aussi le type pour evité certaine erreur au moment des merge 

In [134]:
place = df_place[['id','is_closed','is_published','has_offers','has_food','has_terrace','slug','city_id']]
place=place.astype(object)
place=generate_int_id(place,"place_id_light") # genere des id unique,consecutif de zero a la dernier personne indispensable pour lightfm

on cherche a savoir qu'elle utilisateur on un favoris ou un lien avec au moins un  bar et on va créé une rellation pour les user qui n'en non aucun actuellement 

pour cela on recupere la liste des user dans favoris et on drop les doublon 
cela nous permet de recupere avec un merge qui n'a aucun favoris 

puis on fais un merge entre tous nos user et ceux posssedant une favoris, on ajoutant un indicateur ce qui va permetre d'isolé les user ne possedant pas de favoris car il auront l'indicateur 'left_only' la ou les autre on 'both'

on a donc une liste et un df qui serviront un peut plus tard 

In [135]:
# 
qui_a_un_favoris=df_favorite.drop_duplicates(subset='user_id')
qui_a_un_favoris=qui_a_un_favoris['user_id'] #liste des user ayant au moins un favoris


user_without_favori=user.merge(
    qui_a_un_favoris,how='outer',on='user_id',indicator=True
)#on merge tous nos user avec ceux ayant un favoris en diferentiens ceux ayant les deux par l'indiacteur 

indexNames=user_without_favori[user_without_favori['_merge'] == 'both'].index
user_without_favori.drop(indexNames , inplace=True) # on drop ceux ayant deja un favoris 

list_user_without_favoris=user_without_favori['user_id']#format list des user sans favoris
df_user_without_favoris=user_without_favori[['user_id']]#format dataframe des user sans favoris



pour des soucis explicité en detaille dans notre rapport il faut que tous nos user soit liée a un bar (par une visite,favoris ou review idéalement) mais certains user n'utilise pas ces fonctionalité la dans l'app (choses qui va changer avec le temps)

donc grace a la list des user sans favoris crée juste avant on crée au moins un lien entre chaque user et bar de facons aleatoire (user n'ayant pas de rellation evidement et comme le modele ce rentraineras regulierement cette aleatoire ne serra que temporaire et cela ne derange pas schlouk map d'un point de vue buisness

In [136]:
place_id_list=place['id']#on recupere l'id des bar

liste_user=[]
for i in  list_user_without_favoris: #creation d'une liste avec juste les id des user sans
    liste_user.append(i)

liste_place=[]
for i in range(len(liste_user)): #creation d'une liste d'id de bar de la même longeur que celle des user 
    liste_place.append(random.choice(place_id_list))#choix d'un bar random


dico_user_bar={'user_id':liste_user,'place_id':liste_place}
favorite_for_new_user = pd.DataFrame(dico_user_bar)#creation d'un dataframe des nouvelle rellation user bar
favorite_for_new_user


,user_id,place_id
0,31280,5761
1,33287,4663
2,29924,5446
3,11040,5216
4,32661,3799
...,...,...
40030,8261,7028
40031,16737,6857
40032,39028,4365
40033,40048,5970


maintenant nous pouvons recollé les vrais favorie avec nos favoris artificielle 

In [137]:
favorit_generalle=pd.concat([df_favorite,favorite_for_new_user])
favorit_generalle.nunique()
#on remarque que certaint bar ne ce voit attribué personne ce qui posse probleme

id            15285
place_id       7321
user_id       45480
created_at    15268
dtype: int64

ici on fais le chemins inverse, on determine qu'elle bar n'a pas recue de favoris par des user  et on leur en attribue une mais que sur les user qui n'en n'avais pas initiallement pour ne pas alteré la data des user en possedant 

In [138]:
bar_sans_favoris=favorit_generalle.drop_duplicates(subset='place_id')
bar_sans_favoris=bar_sans_favoris['place_id']#liste des bar possedant au moins une rellations


place_without_favoris = place.merge(
    bar_sans_favoris,how='outer',left_on='id',right_on='place_id',indicator=True
)
indexNames=place_without_favoris[place_without_favoris['_merge'] == 'both'].index
place_without_favoris.drop(indexNames , inplace=True)
#ici on utilise la meme technique que pour user decrite un peut plus haut pour identifier
#les bar sans favoris 

place_without_favoris=place_without_favoris[['id']]

#mise au format de list des user sans favoris 
user_id_list=df_user_without_favoris.user_id.to_list()

#on genere une rellation aleatoire pour les bar qui sont encore non liées
place_without_favoris=place_without_favoris.id.to_list()

list_user=[]
#on genere une rellation aleatoire pour les bar qui sont encore non liées 
# mais seulement avec les user qui n'en non pas initiallement
for i in place_without_favoris:
    list_user.append(random.choice(user_id_list))

dickos={'user_id':list_user,'place_id':place_without_favoris}
favorite_for_place_alone = pd.DataFrame(dickos)

#cette ligne permet de verifier qu'on a bien ajouté le nombre de bar restant vue precedement 
favorite_for_place_alone.nunique()

user_id     28
place_id    28
dtype: int64

et on assemble tous ces favoris crée au favoris generé

In [139]:
favorite=pd.concat([favorit_generalle,favorite_for_place_alone])#simple concat
favorite.nunique()#affiche le nombre de user et place differente (crée & rélle) et qu'il 
#correspond a nos table user et place en longeur 

id            15285
place_id       7349
user_id       45480
created_at    15268
dtype: int64

ici on crée le detaframe globale qui stockera les feature, le poid, et les rellation entre les user et les bar 

In [140]:
# on crée des id unique pour chaque rellation favoris entre les user et place qui 
# respecte les besoin de lightfm 
favorite_for_merge=favorite[['user_id',"place_id"]]
favorite_for_merge = generate_int_id(favorite_for_merge,'favorite_id_light')

# on merge place_type avec place_place_type = type_for_merge
type_for_merge=place_type.merge(
    df_place_place_type,how='inner',right_on='type_id',left_on='id'
)

# on merge place avec type for merge pour assossier les item aux places=place_for_merge
type_for_merge=type_for_merge.drop(["id",'type_id'],axis=1)

place_for_merge=place.merge(
    type_for_merge,how='left',left_on='id',right_on='place_id'
)

# on merge place_for_merge avec les user
df_merge=user.merge(
    favorite_for_merge,how="inner",left_on='user_id',right_on='user_id'
)
#on merge finalement toute nos donné ensemble
df_merge=df_merge.merge(
    place_for_merge,how='left',left_on='place_id',right_on='id'
)




ici on prepare les item necessaire pour lightfm en ce bassant sur les type des bar que les user on en favoris ou autre 

cela permet le fonctionement de type content-based filtering de notre model

et on crée une seule ligne par utilisateur avec les type des bar qu'il a frequenté en decoupant ces dernier un par un pour l'analyse textuel futur du model 

In [141]:
#on recupere de notre df_merge les id light des user et name qui est le type des bar 
# avec les qu'elle ils on interagis 
user_id_and_placetype=df_merge[['user_id_light','name']]

user_id_and_placetype=user_id_and_placetype.dropna()#on drop les na par securité 

#on regroupe les type des bar fréquente par chaque user en une seule ligne
# en les regroupant un a un 
user_id_and_placetype=user_id_and_placetype.groupby(
    ['user_id_light'])['name'].apply(
        ','.join).reset_index()
user_id_and_placetype['name'] = (
    user_id_and_placetype['name'].str.split(',').apply(set).str.join(','))

#on regroupe cela avec les infos que l'on a sur chaque utilisateur pour crée notre  
# dataframe qui possede une ligne par user 
df_user_ready=user.merge(
    user_id_and_placetype,how='left',on='user_id_light'
)

In [142]:
df_user_ready

,user_id,firebase_uid,user_id_light,name
0,31280,zzzZClWPVSQ22uT2SqQqQQfSPon2,0,NaN
1,33287,ZZzYwUhcuTN7WUlKTNVJ0El7Gz83,1,NaN
2,29924,zZZMZBYZKXP3ClI83zvehUXXXsp2,2,NaN
3,11040,ZzZ4AF9HvmaWo3ssjW3vAmgEacj1,3,NaN
4,32661,zzxJ4S3MRxVDxcZ71UWQv7qfUG92,4,NaN
...,...,...,...,...
45475,16737,00KjtvKZG3ahSKyCt3AyRyTAesG3,45475,NaN
45476,31535,00FyNO18CYSV8sLQgbpKH0kQRmP2,45476,NaN
45477,39028,00e24es9Rse3DSsByF9kgR8TPHD3,45477,NaN
45478,40048,00AUdltem0aniOvFw8iRpCgO9Fo1,45478,Bar LGBTQI+


comme pour les user juste avant nous reorganison comment sont stocker les type des bar avec en sortie une seul ligne par bar et s'est type rangé dans une liste

In [143]:
place_for_item= place_for_merge
place_for_item['name']=place_for_item['name'].fillna('No Tag')#pour evité des probleme on remplace les type manquant par no tag 

place_for_item=place_for_item.groupby(
    ['place_id_light'])['name'].apply(
        ','.join).reset_index()
place_for_item['name'] = (
    place_for_item['name'].str.split(',').apply(set).str.join(','))

#on regroupe toute les infos des bar ici
df_place_ready = place.merge(
    place_for_item,how='left',on='place_id_light'
)


on crée les route interne pour le mapping de lightfm partie critique car si un user ou un bar venais a manqué cela crée une discontinuité pour lightfm qui nous retournerais une erreur 

In [144]:
#on genere les list de feature pour le mapping de lightfm
user_feature_list = generate_feature_list(
    df_user_ready,['name']
)
place_feature_list = generate_feature_list(
    df_place_ready,['name']
)


les feature sont les type des bar 
on les associe a chaque user en fonction de sont id light


In [145]:
#creation des feature qui vont allimenté notre modele
df_user_ready['user_feature2']=create_features(
    df_user_ready,['name'],'user_id_light'
)

df_place_ready['place_feature'] = create_features(
    df_place_ready,['name'],'place_id_light'
)

In [146]:
# df_merge['total_weights']=random.randint(0,5)

ici on definie les variable de notre dataset
pour cela on donne l'id unique de chaque user et item(bar/place)
mais aussi les feature tous cela va crée le mapping interne pour notre modele

In [147]:
dataset = Dataset()
dataset.fit(
    set(df_user_ready['user_id_light']),
    set(df_place_ready['place_id_light']), 
    user_features=user_feature_list,
    item_features=place_feature_list)



maintenant on crée la matrice des interaction entre les user et les place
pour cela on passe les id de ces dernier comme des tuple 

et avec cela on utilise la fonction native de lightfm pour crée notre matrice

In [148]:
df_merge['bar_user_id_tuple']=list(zip(
    df_merge.user_id_light,df_merge.place_id_light
))
interactions,weights=dataset.build_interactions(
    df_merge['bar_user_id_tuple']
)

ici on fais construire les feature des bar et user d'une facon que lightfm comprend
pour cela on utilise la fonction integre de lightfm

In [149]:
user_feature = dataset.build_user_features(
    df_user_ready['user_feature2']
)
place_feature = dataset.build_item_features(
    df_place_ready['place_feature']
)

ici on crée notre model avec c'est multiple parametre puis on l'entraine en lui donnant toute les liste et dataframe crée precedement 


attention il faut ajusté le 'num_threads' au nombre de core de votre ordinateur

In [150]:
model = LightFM(
    no_components=300,
    learning_rate=0.02,
    loss='warp',
    random_state=2019)

model.fit(
    interactions,
    user_features=user_feature,
    item_features=place_feature,
    sample_weight=weights,
    epochs=7, num_threads=8, verbose=True)

Epoch: 100%|██████████| 7/7 [00:06<00:00,  1.14it/s]


on calcule le score AUC ici et obtient 0,75 ce qui n'est pas mauvais au vue du manque de data de notre situation 

a noté que avec le temps la db de schlouk map va grandir et donc permettre a notre modele de devenir meilleur 

In [151]:
calculate_auc_score(model,interactions,place_feature,user_feature)

0.8407551

on extrait les id des user bar et item du mapping de notre dataset pour pouvoir les ciblé pendant nos prediction

In [152]:
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()

n_users, n_items = interactions.shape# permet d'avoir la taille de notre dataset et 
#ne pas le depassé pendant les prediction

In [164]:
recommandation_for_user_by_city(2,3)

,index,score,id,is_closed,is_published,has_offers,has_food,has_terrace,slug,city_id,place_id_light
3863,3863,0.511049,4032,0,1,0,1.0,1.0,le-viaduc,3,3863
3488,3488,0.395485,3655,0,1,0,1.0,1.0,le-montebello,3,3488
3823,3823,0.279922,3992,0,1,0,1.0,1.0,cabana-beach,3,3823
3528,3528,0.263438,3696,0,1,0,NaN,NaN,team-brothers,3,3528
512,512,0.248808,520,0,1,0,1.0,1.0,dalea,3,512
...,...,...,...,...,...,...,...,...,...,...,...
401,401,-0.620456,408,0,1,0,NaN,NaN,au-fut-et-a-mesure-1,3,401
833,833,-0.647229,848,0,1,0,1.0,NaN,lizard-lounge,3,833
3520,3520,-0.651768,3687,0,1,0,1.0,1.0,cicchetti-1,3,3520
430,430,-0.652812,437,0,1,0,NaN,NaN,le-onze,3,430


notre fonction definie dans le bloc des fonction qui retourne les bar trié par pertinance pour un user en filtrant selon la ville et les bar qu'il connais deja 

In [154]:
b=time.perf_counter()
print(b-a)

131.82801504200006
